In [6]:
%pip install -q dagshub mlflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.0/262.0 kB 7.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 80.3 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 75.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.2/131.2 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.4/797.4 kB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import mlflow
import dagshub
dagshub.init(repo_owner='reddyrohith49471', repo_name='amazon-ml-2025', mlflow=True)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

Output()



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=e7e3e238-9d04-4307-9e7f-4069bab34489&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=e66c95f9dfd7e5740e40871dd69619151fe969fec4bee907e7e54ac879da1dd6




Accessing as reddyrohith49471

Initialized MLflow to track repo "reddyrohith49471/amazon-ml-2025"

Repository reddyrohith49471/amazon-ml-2025 initialized!

In [8]:
train = pd.read_csv("/kaggle/input/trainml/train.csv")

In [9]:
train.shape

(75000, 19)

In [10]:
train.head(1)

,catalog_content,image_link,price,has_item_name,has_value,has_unit,Item Name,Value,unit,num_bullets,bullet_1,bullet_2,bullet_3,bullet_4,bullet_5,combined_text,price_bin,price_clipped,price_log
0,"Item Name: La Victoria Green Taco Sauce Mild, ...",https://m.media-amazon.com/images/I/51mo8htwTH...,4.89,True,True,True,"La Victoria Green Taco Sauce Mild, 12 Ounce (P...",72.0,Fl Oz,0,NaN,NaN,NaN,NaN,NaN,"La Victoria Green Taco Sauce Mild, 12 Ounce (P...",$0-$200,4.89,1.773256


In [11]:
X = train[["combined_text","Value"]]
Y = train["price_log"]

In [12]:
X_train, X_test, Y_train_price_log, Y_test_price_log, Y_train_price, Y_test_price = train_test_split(X,
                                                          train["price_log"],
                                                          train["price"],
                                                          test_size=0.2,
                                                          random_state=42)

In [13]:
print(X_train.shape)
print(X_test.shape)
print(Y_train_price_log.shape)
print(Y_test_price_log.shape)
print(Y_train_price.shape)
print(Y_test_price.shape)


(60000, 2)
(15000, 2)
(60000,)
(15000,)
(60000,)
(15000,)


In [14]:
text_col = "combined_text"
num = ["Value"]
preprocessor = ColumnTransformer(
    transformers=[
        ("text",TfidfVectorizer(
            max_features=5000,
            ngram_range=(1,2),
            min_df=5,
            stop_words="english"
        ),text_col),
        ("num",StandardScaler(),num)
    ]
)

In [15]:
def decode_price(price_log_pred, upper_limit=200):
    price_clipped_pred = np.expm1(price_log_pred)
    price_pred = np.clip(price_clipped_pred, None, upper_limit)
    return price_pred


In [16]:
def smape(y_true, y_pred):
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0
    return np.mean(diff) * 100

In [17]:
def run_experiment(model,run_name):
    with mlflow.start_run(run_name=run_name):
        #Train
        model.fit(X_train,Y_train_price_log)

        #Predict
        y_pred_log = model.predict(X_test)
        y_pred_price = decode_price(y_pred_log,upper_limit=200)

        #Metrics
        mse = mean_squared_error(y_pred_price,Y_test_price)
        smape_score = smape(y_pred_price,Y_test_price)
        rmse = np.sqrt(mse)

        #Logging Params
        mlflow.log_param("model", model.named_steps["regressor"].__class__.__name__)
        mlflow.log_param("text_column","combined_text")
        mlflow.log_param("numeric_column","Value")
        mlflow.log_param("max_features",5000)
        mlflow.log_param("ngram_range","(1,2)")

        #Logging Metrics
        mlflow.log_metric("mse", mse)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("smape", smape_score)

        #Logging Model
        mlflow.sklearn.log_model(model,"model")
    scores = {"MSE:":mse,
             "RMSE:":rmse,
             "SMAPE:":smape_score}
    return scores
        

In [18]:
model = Pipeline([
    ("preprocessor",preprocessor),
    ("regressor",LinearRegression())
])
run_experiment(model,"Linear_Regression_Baseline")

2026/02/10 08:56:27 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/usr/local/lib/python3.12/dist-packages/mlflow/models/model.py:1209: FutureWarning: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization.The recommended safe alternative is the 'skops' format.
  flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)


🏃 View run Linear_Regression_Baseline at: https://dagshub.com/reddyrohith49471/amazon-ml-2025.mlflow/#/experiments/0/runs/8ea34c284ade40bb8826388c2f4b850a
🧪 View experiment at: https://dagshub.com/reddyrohith49471/amazon-ml-2025.mlflow/#/experiments/0


{'MSE:': 1247.7861741818538,
 'RMSE:': np.float64(35.324016959879486),
 'SMAPE:': np.float64(58.910081187522955)}